<a href="https://colab.research.google.com/github/bharath637/python/blob/main/pyspark_practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip3 install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 17.6 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=bd1724bbb9ef11cae9f6fbd908aa3c2dfabea8d7350ddd472ddc12fca401dd1f
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark


In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
spark

In [ ]:
from pyspark.sql.functions import collect_list
data = [ (1,"hotstar"),(1,"netflix"),(2,"hotstar"),(3,"netflix"),(4,"zee"),(4,"netflix"),(4,"hotstar")]
schema = ["ID","SERVICE"]

df = spark.createDataFrame(data,schema)
df.show()





+---+-------+
| ID|SERVICE|
+---+-------+
|  1|hotstar|
|  1|netflix|
|  2|hotstar|
|  3|netflix|
|  4|    zee|
|  4|netflix|
|  4|hotstar|
+---+-------+



In [ ]:
from pyspark.sql.functions import col,lit,concat_ws,when
df1=df.groupBy('ID').agg(collect_list(col("SERVICE")).alias("all_services")).select(col("ID").alias("no"),concat_ws(',',col("all_services")))
df1.show()

In [ ]:
#Prepare column list
from datetime import date
date="2022-01-21"
col_list = [eval('df.%s' %cl) for cl in df.columns]
print(col_list)
df2= df.select(col_list).withColumn("ld_dt",lit(date))
df2.show()

[Column<'ID'>, Column<'SERVICE'>]
+---+-------+----------+
| ID|SERVICE|     ld_dt|
+---+-------+----------+
|  1|hotstar|2022-01-21|
|  1|netflix|2022-01-21|
|  2|hotstar|2022-01-21|
|  3|netflix|2022-01-21|
|  4|    zee|2022-01-21|
|  4|netflix|2022-01-21|
|  4|hotstar|2022-01-21|
+---+-------+----------+



In [ ]:
# use when and otherwise
df.show()
df.select(col("ID"),when(col("SERVICE")=='hotstar',"HotStar Premium").otherwise(col("SERVICE")).alias("SERVICE")).show()


+---+-------+
| ID|SERVICE|
+---+-------+
|  1|hotstar|
|  1|netflix|
|  2|hotstar|
|  3|netflix|
|  4|    zee|
|  4|netflix|
|  4|hotstar|
+---+-------+

+---+---------------+
| ID|        SERVICE|
+---+---------------+
|  1|HotStar Premium|
|  1|        netflix|
|  2|HotStar Premium|
|  3|        netflix|
|  4|            zee|
|  4|        netflix|
|  4|HotStar Premium|
+---+---------------+



In [ ]:
data = [ (5,"hotstar"),(5,"netflix"),(6,"hotstar"),(6,"netflix"),(7,"zee"),(8,"netflix"),(9,"hotstar")]
schema = ["ID","SERVICE"]

df5 = spark.createDataFrame(data,schema)
#df5.show()

# UNION ALL
from pyspark.sql import DataFrame
from functools import reduce
def unionall(*datafra):
  return reduce(DataFrame.unionByName,datafra)
final_df = unionall(*[df,df5])
final_df.show()


+---+-------+
| ID|SERVICE|
+---+-------+
|  1|hotstar|
|  1|netflix|
|  2|hotstar|
|  3|netflix|
|  4|    zee|
|  4|netflix|
|  4|hotstar|
|  5|hotstar|
|  5|netflix|
|  6|hotstar|
|  6|netflix|
|  7|    zee|
|  8|netflix|
|  9|hotstar|
+---+-------+



In [ ]:
def summ(*inp):
  return reduce(lambda x,y:x+y,inp)

sum([1,2,4,5,5])




17

In [10]:
#Remove special characters

df11 = spark.sql("select '&=()' as matchstring")
df11.show()
matchstring = df11.collect()[0].matchstring
data2 = [ (10,"hotstar&"),(11,"netflix=")]
schema2 = ["ID","SERVICE"]
df12 = spark.createDataFrame(data2,schema2)
from pyspark.sql.functions import translate,col,trim
df12 =df12.select(trim(translate(col("SERVICE"),matchstring," ")))
df12.show()


+-----------+
|matchstring|
+-----------+
|       &=()|
+-----------+

+---------------------------------+
|trim(translate(SERVICE, &=(),  ))|
+---------------------------------+
|                          hotstar|
|                          netflix|
+---------------------------------+



Row(matchstring='&=()')